In [1]:
#import desired packages
import tensorflow as tf
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os.path
import sys
# library for freezing the graph
from tensorflow.python.tools import freeze_graph
# library for optmising inference
from tensorflow.python.tools import optimize_for_inference_lib
%matplotlib inline 

C:\Users\josepablo\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
#import data 
data=pd.read_csv('vinos.csv', names=['f1','f2','f3','f4','f5',"f6","f7","f8","f9","f10","f11","f12","f13","f14"])


#map data into arrays
s=np.asarray([1,0,0])
ve=np.asarray([0,1,0])
vi=np.asarray([0,0,1])
data['f14'] = data['f14'].map({'vino1': s, 'vino2': ve,'vino3':vi})


#shuffle the data
data=data.iloc[np.random.permutation(len(data))]
data=data.reset_index(drop=True)


#training data
x_input=data.loc[0:105,['f1','f2','f3','f4','f5','f6','f7','f8','f9','f10','f11','f12','f13']]
y_input=data['f14'][0:106]

#test data
x_test=data.loc[106:177,['f1','f2','f3','f4','f5','f6','f7','f8','f9','f10','f11','f12','f13']]
y_test=data['f14'][106:178]

In [3]:
data

,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14
0,13.17,5.19,2.32,22.0,93,1.74,0.63,0.61,1.55,7.900000,0.600,1.48,725,"[0, 0, 1]"
1,11.41,0.74,2.50,21.0,88,2.48,2.01,0.42,1.44,3.080000,1.100,2.31,434,"[0, 1, 0]"
2,13.11,1.90,2.75,25.5,116,2.20,1.28,0.26,1.56,7.100000,0.610,1.33,425,"[0, 0, 1]"
3,12.29,1.41,1.98,16.0,85,2.55,2.50,0.29,1.77,2.900000,1.230,2.74,428,"[0, 1, 0]"
4,13.40,3.91,2.48,23.0,102,1.80,0.75,0.43,1.41,7.300000,0.700,1.56,750,"[0, 0, 1]"
5,12.47,1.52,2.20,19.0,162,2.50,2.27,0.32,3.28,2.600000,1.160,2.63,937,"[0, 1, 0]"
6,13.32,3.24,2.38,21.5,92,1.93,0.76,0.45,1.25,8.420000,0.550,1.62,650,"[0, 0, 1]"
7,13.62,4.95,2.35,20.0,92,2.00,0.80,0.47,1.02,4.400000,0.910,2.05,550,"[0, 0, 1]"
8,13.50,1.81,2.61,20.0,96,2.53,2.61,0.28,1.66,3.520000,1.120,3.82,845,"[1, 0, 0]"
9,13.05,2.05,3.22,25.0,124,2.63,2.68,0.47,1.92,3.580000,1.130,3.20,830,"[1, 0, 0]"


In [4]:
#placeholders and variables. input has 4 features and output has 3 classes
x=tf.placeholder(tf.float32,shape=[None,13] , name="Input")
y_=tf.placeholder(tf.float32,shape=[None, 3])
#weight and bias
W=tf.Variable(tf.zeros([13,3]))
b=tf.Variable(tf.zeros([3]))

# model 
#softmax function for multiclass classification
y = tf.nn.softmax((tf.matmul(x, W) + b),name="output")

# Cost function
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))

# Optimiser 
train_step = tf.train.AdamOptimizer(0.01).minimize(cross_entropy)
#calculating accuracy of our model 
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
output=tf.argmax(y,axis=1)

#session parameters
sess = tf.InteractiveSession()
#initialising variables
init = tf.global_variables_initializer()
sess.run(init)
#number of interations
epoch=2000

In [5]:
#Training 
for step in range(epoch):
    _, loss=sess.run([train_step,cross_entropy], feed_dict={x: x_input, y_:[t for t in y_input.as_matrix()]})
    if step%500==0 :
        print (loss)

C:\Users\josepablo\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  This is separate from the ipykernel package so we can avoid doing imports until


1.0986122
0.10494264
0.06869027
0.05121682


In [6]:
# grabbing the default graph
g = tf.get_default_graph()

# every operations in our graph
[op.name for op in g.get_operations()]

['Input',
 'Placeholder',
 'zeros',
 'Variable',
 'Variable/Assign',
 'Variable/read',
 'zeros_1',
 'Variable_1',
 'Variable_1/Assign',
 'Variable_1/read',
 'MatMul',
 'add',
 'output',
 'Log',
 'mul',
 'Sum/reduction_indices',
 'Sum',
 'Neg',
 'Const',
 'Mean',
 'gradients/Shape',
 'gradients/grad_ys_0',
 'gradients/Fill',
 'gradients/Mean_grad/Reshape/shape',
 'gradients/Mean_grad/Reshape',
 'gradients/Mean_grad/Shape',
 'gradients/Mean_grad/Tile',
 'gradients/Mean_grad/Shape_1',
 'gradients/Mean_grad/Shape_2',
 'gradients/Mean_grad/Const',
 'gradients/Mean_grad/Prod',
 'gradients/Mean_grad/Const_1',
 'gradients/Mean_grad/Prod_1',
 'gradients/Mean_grad/Maximum/y',
 'gradients/Mean_grad/Maximum',
 'gradients/Mean_grad/floordiv',
 'gradients/Mean_grad/Cast',
 'gradients/Mean_grad/truediv',
 'gradients/Neg_grad/Neg',
 'gradients/Sum_grad/Shape',
 'gradients/Sum_grad/Size',
 'gradients/Sum_grad/add',
 'gradients/Sum_grad/mod',
 'gradients/Sum_grad/Shape_1',
 'gradients/Sum_grad/range/sta

In [7]:
saver = tf.train.Saver()
model_directory='model_files/'
if not os.path.exists(model_directory):
        os.makedirs(model_directory)
#saving the graph
tf.train.write_graph(sess.graph_def, model_directory, 'savegraph.pbtxt')    

'model_files/savegraph.pbtxt'

In [8]:
# saving the values of weights and other parameters of the model
saver.save(sess, 'model_files/model.ckpt')

'model_files/model.ckpt'

In [9]:
# Freeze the graph
MODEL_NAME = 'vino'
input_graph_path = 'model_files/savegraph.pbtxt'
checkpoint_path = 'model_files/model.ckpt'
input_saver_def_path = ""
input_binary = False
output_node_names = "output"
restore_op_name = "save/restore_all"
filename_tensor_name = "save/Const:0"
output_frozen_graph_name = 'model_files/frozen_model_'+MODEL_NAME+'.pb'
output_optimized_graph_name = 'model_files/optimized_inference_model_'+MODEL_NAME+'.pb'
clear_devices = True

In [10]:
freeze_graph.freeze_graph(input_graph_path, input_saver_def_path,
                          input_binary, checkpoint_path, output_node_names,
                          restore_op_name, filename_tensor_name,
                          output_frozen_graph_name, clear_devices, "")

INFO:tensorflow:Restoring parameters from model_files/model.ckpt
INFO:tensorflow:Froze 2 variables.
INFO:tensorflow:Converted 2 variables to const ops.


In [11]:
output_graph_def = optimize_for_inference_lib.optimize_for_inference(
        sess.graph_def,
        ["Input"], # an array of the input node(s)
        ["output"], # an array of output nodes
        tf.float32.as_datatype_enum)

In [12]:
output_graph_def

node {
  name: "Input"
  op: "Placeholder"
  attr {
    key: "dtype"
    value {
      type: DT_FLOAT
    }
  }
}
node {
  name: "Variable"
  op: "VariableV2"
  attr {
    key: "container"
    value {
      s: ""
    }
  }
  attr {
    key: "dtype"
    value {
      type: DT_FLOAT
    }
  }
  attr {
    key: "shape"
    value {
      shape {
        dim {
          size: 13
        }
        dim {
          size: 3
        }
      }
    }
  }
  attr {
    key: "shared_name"
    value {
      s: ""
    }
  }
}
node {
  name: "Variable_1"
  op: "VariableV2"
  attr {
    key: "container"
    value {
      s: ""
    }
  }
  attr {
    key: "dtype"
    value {
      type: DT_FLOAT
    }
  }
  attr {
    key: "shape"
    value {
      shape {
        dim {
          size: 3
        }
      }
    }
  }
  attr {
    key: "shared_name"
    value {
      s: ""
    }
  }
}
node {
  name: "MatMul"
  op: "MatMul"
  input: "Input"
  input: "Variable"
  attr {
    key: "T"
    value {
      type: DT_

In [13]:
with tf.gfile.GFile(output_optimized_graph_name, "wb") as f:
            f.write(output_graph_def.SerializeToString())

In [14]:
g = tf.GraphDef()
##checking frozen graph
g.ParseFromString(open(output_optimized_graph_name, 'rb').read())
g

node {
  name: "Input"
  op: "Placeholder"
  attr {
    key: "dtype"
    value {
      type: DT_FLOAT
    }
  }
}
node {
  name: "Variable"
  op: "VariableV2"
  attr {
    key: "container"
    value {
      s: ""
    }
  }
  attr {
    key: "dtype"
    value {
      type: DT_FLOAT
    }
  }
  attr {
    key: "shape"
    value {
      shape {
        dim {
          size: 13
        }
        dim {
          size: 3
        }
      }
    }
  }
  attr {
    key: "shared_name"
    value {
      s: ""
    }
  }
}
node {
  name: "Variable_1"
  op: "VariableV2"
  attr {
    key: "container"
    value {
      s: ""
    }
  }
  attr {
    key: "dtype"
    value {
      type: DT_FLOAT
    }
  }
  attr {
    key: "shape"
    value {
      shape {
        dim {
          size: 3
        }
      }
    }
  }
  attr {
    key: "shared_name"
    value {
      s: ""
    }
  }
}
node {
  name: "MatMul"
  op: "MatMul"
  input: "Input"
  input: "Variable"
  attr {
    key: "T"
    value {
      type: DT_

In [15]:
g1 = tf.GraphDef()
##checking frozen graph
g1.ParseFromString(open("model_files/frozen_model_vino.pb", 'rb').read())
g1

node {
  name: "Input"
  op: "Placeholder"
  attr {
    key: "dtype"
    value {
      type: DT_FLOAT
    }
  }
  attr {
    key: "shape"
    value {
      shape {
        dim {
          size: -1
        }
        dim {
          size: 13
        }
      }
    }
  }
}
node {
  name: "Variable"
  op: "Const"
  attr {
    key: "dtype"
    value {
      type: DT_FLOAT
    }
  }
  attr {
    key: "value"
    value {
      tensor {
        dtype: DT_FLOAT
        tensor_shape {
          dim {
            size: 13
          }
          dim {
            size: 3
          }
        }
        tensor_content: "M\321\261\276\220h\354>\245\221A\274\026n\320>~?U\277Z\354\245>\367\311\234?\377\005\250\277\311w\234\2751\033\236\276`\327S>i\030\030>\356\2304\274@\246\256<\361y\271<\273\362\327\2767\331\264?\026\353A\277{\200\303?\314\000L>&\237\311\277 q\205\276N\357(?\252t\216\276\315T\357=\363\177\036?r\332\035\277E\326\362=*.\261\277\210-\203?\242\273a\277\032\232\t@2\227\203\277\343\313K?YI*?5V

In [16]:
g1==g

False